# Import packages

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
import matplotlib
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import scanpy as sc
import matplotlib.pyplot as plt
from natsort import natsorted
from itertools import combinations
from scipy.stats import ttest_ind, f_oneway
from statsmodels.stats.multitest import multipletests

import OncoMarkAI as oma

# Plot setting

In [ ]:
oma.pl.fig_setting()
cmap = oma.pl.cmap()
sns.set_style("ticks");
sns.despine(offset=10, trim=True);

In [ ]:
## Read HLAs
HLAs = pd.read_csv("../OncoMarkAI/data/datasets/OptiTypeCallsHLA_20171207.tsv", index_col=8)
HLAs.index = ["-".join(idx.split("-")[:3]) for idx in HLAs.index]
HLAs = HLAs.iloc[:, :6]
HLAs = HLAs.rename_axis("submitter_id").reset_index().drop_duplicates().set_index("submitter_id")
HLAs["A"] = [[a1.split(":")[0], a2.split(":")[0]] for a1, a2 in HLAs[["A1", "A2"]].values]

In [ ]:
frac = HLAs[["A"]].explode("A").reset_index().drop_duplicates().groupby("A").size().sort_values(ascending=False) / HLAs.index.nunique()
frac = frac.to_frame("percentage")
frac["%"] = round(frac*100, 1)
frac["allotypes"] = frac.index.astype(str) + " (" + frac["%"].astype(str) + "%)"

In [ ]:
frac

In [ ]:
patient_data_TCGA_tumor = oma.data.load_tcga(file_path="../OncoMarkAI/data/datasets/TCGA_log2TPMplus1_protein_coding_transcripts_20221113.h5ad", 
                                             metadata="../OncoMarkAI/data/datasets/combined_study_clinical_data_cBioPortal.tsv",
                                             filter_samples=100, sample_type="Tumor", genes=["IGSF8", "KIR3DL2"])
IGSF8_tumor = patient_data_TCGA_tumor.to_df().join(patient_data_TCGA_tumor.obs[["project_id","submitter_id", "samples.sample_type"]])
IGSF8_tumor = IGSF8_tumor[~IGSF8_tumor['project_id'].isin(['LAML', "DLBC"])].set_index("submitter_id")

In [ ]:
HLA_A_IGSF8 = HLAs[["A"]].explode("A").join(IGSF8_tumor).dropna()
HLA_A_IGSF8["A"] = [a.split(":")[0] for a in HLA_A_IGSF8["A"]]
HLA_A_IGSF8 = HLA_A_IGSF8[HLA_A_IGSF8["A"].isin(frac.index[:10])]
HLA_A_IGSF8["allotypes"] = HLA_A_IGSF8["A"].map(frac["allotypes"])

In [ ]:
HLA_A_IGSF8

In [ ]:
grouped = HLA_A_IGSF8.groupby(['A'])
tstat_results = pd.DataFrame(index=grouped["A"].count().index, columns=["tstat", "pvalue"])
for idx,row in tstat_results.iterrows():
    tstat, p_value = ttest_ind(HLA_A_IGSF8.query('A==@idx')["IGSF8"].values, HLA_A_IGSF8.query('A!=@idx')["IGSF8"].values)
    tstat_results.loc[idx, "tstat"] = tstat
    tstat_results.loc[idx, "pvalue"] = p_value
tstat_results.loc[:, "adj.pval"] = multipletests(tstat_results.loc[:, "pvalue"], method='fdr_bh', alpha=0.05)[1]

In [ ]:
tstat_results

In [ ]:
tstat_results = tstat_results[tstat_results["adj.pval"] < 0.05]
tstat_results

In [ ]:
HLA_A_IGSF8

# Split patients by A*03/11

In [ ]:
## Read HLAs
HLAs = pd.read_csv("../OncoMarkAI/data/datasets/OptiTypeCallsHLA_20171207.tsv", index_col=8)
HLAs.index = ["-".join(idx.split("-")[:3]) for idx in HLAs.index]
HLAs = HLAs.iloc[:, :2]
HLAs["A1"] = [a1.split(":")[0] for a1 in HLAs["A1"].values]
HLAs["A2"] = [a1.split(":")[0] for a1 in HLAs["A2"].values]

HLAs = HLAs.rename_axis("submitter_id").reset_index().drop_duplicates().set_index("submitter_id")
HLAs["A"] = [natsorted([a1, a2]) if a1!=a2 else [a1]
             for a1, a2 in HLAs[["A1", "A2"]].values]
HLAs = HLAs.loc[~HLAs.index.duplicated()]

In [ ]:
HLA_A03_mask = (HLAs[["A1", "A2"]] == "A*03").sum(axis=1)>0
HLA_A11_mask = (HLAs[["A1", "A2"]] == "A*11").sum(axis=1)>0

HLA_A_IGSF8["A*03/11"] = HLA_A_IGSF8.index.map(HLA_A11_mask | HLA_A03_mask)
HLA_A_IGSF8["A*03/11"] = HLA_A_IGSF8["A*03/11"].replace(True, "positive").replace(False, "negative")

In [ ]:
HLA_A_IGSF8 = HLA_A_IGSF8.loc[HLA_A_IGSF8.index.duplicated()]

In [ ]:
HLA_A_IGSF8

In [ ]:
plt.figure(figsize=(3.5,2.2))
ax = sns.boxplot(data=HLA_A_IGSF8,
                 x="project_id",
                 order=HLA_A_IGSF8[["KIR3DL2", "project_id"]].groupby("project_id").median().sort_values("KIR3DL2", ascending=False).index,
                 y="KIR3DL2",
                 hue="A*03/11",
                 hue_order=["negative", "positive"],                 
                 palette={"positive": cmap.npg_palette(0), "negative": cmap.npg_palette(3)},
                 fill=False,
                 linecolor="#137",
                 linewidth=.5,
                 width=.75,
                 gap=0.2,
                 showfliers=False,
                 medianprops={'linewidth': 1},
                )

ax = sns.stripplot(data=HLA_A_IGSF8,
                   x="project_id",
                   order=HLA_A_IGSF8[["KIR3DL2", "project_id"]].groupby("project_id").median().sort_values("KIR3DL2", ascending=False).index,
                   y="KIR3DL2",
                   hue="A*03/11",
                   hue_order=["negative", "positive"],
                   palette={"positive": cmap.npg_palette(0), "negative": cmap.npg_palette(3)},
                   dodge=True,
                   s=1,
                   alpha=.5,
                   jitter=.2,
                   legend=False
                  )

ax.set_xlabel("")
ax.set_ylabel("KIR3DL2 expression log2(TPM+1)", fontsize=5)
ax.legend(loc="upper right", title="HLA-A*03/11 carriers",  title_fontsize=4, fontsize=4, frameon=False) #cols=1,
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.set_ylim(None, 4.5)
plt.savefig("../figures/FigS3U.pdf")

In [ ]:
plt.figure(figsize=(3.5,2.2))
ax = sns.boxplot(data=HLA_A_IGSF8,
                 x="project_id",
                 order=HLA_A_IGSF8[["IGSF8", "project_id"]].groupby("project_id").median().sort_values("IGSF8", ascending=False).index,
                 y="IGSF8",
                 hue="A*03/11",
                 palette={"positive": cmap.npg_palette(0), "negative": cmap.npg_palette(3)},
                 hue_order=["negative", "positive"],
                 fill=False,
                 linecolor="#137",
                 linewidth=.5,
                 width=.75,
                 gap=0.2,
                 showfliers=False,
                 medianprops={'linewidth': 1},
                )

ax = sns.stripplot(data=HLA_A_IGSF8,
                   x="project_id",
                   order=HLA_A_IGSF8[["IGSF8", "project_id"]].groupby("project_id").median().sort_values("IGSF8", ascending=False).index,
                   y="IGSF8",
                   hue="A*03/11",
                   hue_order=["negative", "positive"],
                   palette={"positive": cmap.npg_palette(0), "negative": cmap.npg_palette(3)},
                   dodge=True,
                   s=1,
                   alpha=.5,
                   jitter=.2,
                   legend=False
                  )

ax.set_xlabel("")
ax.set_ylabel("IGSF8 expression log2(TPM+1)", fontsize=5)
ax.legend(loc="upper right", title="HLA-A*03/11 carriers",  title_fontsize=4, fontsize=4, frameon=False) #cols=1,
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.set_ylim(None, 11.5)
plt.savefig("../figures/FigS3V.pdf")

## Survival analysis with HLA allotype stratification

In [ ]:
## Read HLAs
HLAs = pd.read_csv("../OncoMarkAI/data/datasets/OptiTypeCallsHLA_20171207.tsv", index_col=8)
HLAs.index = ["-".join(idx.split("-")[:3]) for idx in HLAs.index]
HLAs = HLAs.iloc[:, :2]
HLAs["A1"] = [a1.split(":")[0] for a1 in HLAs["A1"].values]
HLAs["A2"] = [a1.split(":")[0] for a1 in HLAs["A2"].values]

HLAs = HLAs.rename_axis("submitter_id").reset_index().drop_duplicates().set_index("submitter_id")
HLAs["A"] = [natsorted([a1, a2]) if a1!=a2 else [a1]
             for a1, a2 in HLAs[["A1", "A2"]].values]
HLAs = HLAs.loc[~HLAs.index.duplicated()]

In [ ]:
HLAs[["A"]].explode("A")

In [ ]:
HLA_A03_mask = (HLAs[["A1", "A2"]] == "A*03").sum(axis=1)>0
HLA_A11_mask = (HLAs[["A1", "A2"]] == "A*11").sum(axis=1)>0

In [ ]:
# Read RNAseq data
patient_data_TCGA = oma.data.load_tcga(file_path="../OncoMarkAI/data/datasets/TCGA_log2TPMplus1_protein_coding_transcripts_20221113.h5ad", 
                                       metadata="../OncoMarkAI/data/datasets/combined_study_clinical_data_cBioPortal.tsv",
                                       filter_samples=100,
                                       sample_type="Tumor",
                                       genes=["IGSF8"])

In [ ]:
# generate a dataframe to integrate expression data and metadata
expr_data = patient_data_TCGA.to_df().join(patient_data_TCGA.obs[["project_id", "samples.sample_type", "submitter_id", "cbioportal.subtype", "subtype(MSI)", "subtype(BRCA)", "subtype(SKCM)", "race"]])

# there are a few duplicated samples, likely processed by different analysts, we drop the duplicates and keep only the first entries
expr_data = expr_data[~expr_data["submitter_id"].duplicated()]

In [ ]:
# read the clinical data of TCGA patient samples. Data was obtained from Liu et al, Cell (2018).
CDR = pd.read_excel("../OncoMarkAI/data/datasets/TCGA-CDR_Liu2018.xlsx", index_col=1)

# annotate the stage according to the Liu et al, Cell (2018)
CDR.loc[CDR['type'].isin(["CESC", "DLBC", "OV", "UCEC", "UCS"]), 'stage'] = CDR.loc[CDR['type'].isin(["CESC", "DLBC", "OV", "UCEC", "UCS"]), 'clinical_stage']
CDR.loc[~CDR['type'].isin(["CESC", "DLBC", "OV", "UCEC", "UCS"]), 'stage'] = CDR.loc[~CDR['type'].isin(["CESC", "DLBC", "OV", "UCEC", "UCS"]), 'ajcc_pathologic_tumor_stage']

In [ ]:
# Map the stage info into "early-stage" (1) and "late-stage" (2)
# We can also map it into 1, 2, 3, 4 as it is. The results are generally comparable, but some cohorts with few stage 4 tumors would run into issues.
mapper = {
    "Stage I": 1, "Stage IA": 1, "Stage IA1": 1, "Stage IA2": 1, "Stage IB": 1, "Stage IB1": 1, "Stage IB2": 1, "Stage IC": 1, "I/II NOS": 1,
    "Stage II": 1, "Stage IIA": 1, "Stage IIA1": 1, "Stage IIA2": 1, "Stage IIB": 1, "Stage IIC": 1,
    "Stage III": 2, "Stage IIIA": 2, "Stage IIIB": 2, "Stage IIIC": 2, "Stage IIIC1": 2, "Stage IIIC2": 2,
    "Stage IV": 2, "Stage IVA": 2, "Stage IVB": 2, "Stage IVC": 2
}

In [ ]:
# Map the clinical info into expression data
expr_data["age"] = expr_data["submitter_id"].map(CDR["age_at_initial_pathologic_diagnosis"])
expr_data["gender"] = expr_data["submitter_id"].map(CDR["gender"])
expr_data["race"] = expr_data["submitter_id"].map(CDR["race"])
expr_data["stage"] = expr_data["submitter_id"].map(CDR["stage"].map(mapper))
expr_data["OS"] = expr_data["submitter_id"].map(CDR["OS"])
expr_data["OS.time"] = expr_data["submitter_id"].map(CDR["OS.time"])
expr_data["PFI"] = expr_data["submitter_id"].map(CDR["PFI"])
expr_data["PFI.time"] = expr_data["submitter_id"].map(CDR["PFI.time"])
expr_data["race"] = expr_data["race"].replace("[Unknown]", np.nan).replace("[Not Evaluated]", np.nan).replace("[Not Available]", np.nan)

In [ ]:
# Add HLA-A*11 mask
expr_data["A03"] = expr_data["submitter_id"].map(HLA_A03_mask)
expr_data["A11"] = expr_data["submitter_id"].map(HLA_A11_mask)
expr_data["A03/11"] = expr_data["submitter_id"].map(HLA_A11_mask | HLA_A03_mask) # Split patients into A03/A11 carrier or A03/11 non-carrier

In [ ]:
def divide_samples(df_input, divideby='IGSF8', cutoff='mean'):
    """
    divide TCGA samples by the `divideby` parameter according the parameter of `cutoff`
    """
    df = df_input.copy()
    if cutoff=="mean":
        df[f'{divideby}_group'] = np.sign(df[divideby] - df[divideby].mean())
        df[f'{divideby}_group'] = df[f'{divideby}_group'].mask(df[f'{divideby}_group'] ==1, "high")
        df[f'{divideby}_group'] = df[f'{divideby}_group'].mask(df[f'{divideby}_group'] ==-1, "low")

    elif cutoff=="median":
        df[f'{divideby}_group'] = np.sign(df[divideby] - df[divideby].median())
        df[f'{divideby}_group'] = df[f'{divideby}_group'].mask(df[f'{divideby}_group'] ==1, "high")
        df[f'{divideby}_group'] = df[f'{divideby}_group'].mask(df[f'{divideby}_group'] ==-1, "low")

    elif float(cutoff) < 1 and float(cutoff) >= 0.5:     

        cutoff1 = df[divideby].quantile(q=float(cutoff))
        cutoff2 = df[divideby].quantile(q=1-float(cutoff))

        df.loc[:, f'{divideby}_group'] = "intermediate"
        df.loc[:, f'{divideby}_group'] = df[f'{divideby}_group'].mask(df[divideby]>cutoff1, "high").copy()
        df.loc[:, f'{divideby}_group'] = df[f'{divideby}_group'].mask(df[divideby]<cutoff2, "low").copy()
        
    elif cutoff == 1:

        df.loc[:, f'{divideby}_group'] = "high"
        df.loc[:, f'{divideby}_group'] = df[f'{divideby}_group'].mask(df[divideby]!=cutoff, "low").copy()

    high = df[df[f'{divideby}_group'] == "high"]
    low = df[df[f'{divideby}_group'] == "low"]
    
    return high, low, df

In [ ]:
def get_four_subpopulations(input_df, divideby=["A03/11", "IGSF8"], cutoff=[1, ".75"]):
    """
    A helper function to divide patient samples into four subgroups
    """
    survival_subdata = input_df[input_df["OS.time"].notna()]
    div1_high, div1_low, _ = divide_samples(survival_subdata, divideby=divideby[0], cutoff=cutoff[0])
    
    div1_high_div2_high, div1_high_div2_low, _ = divide_samples(div1_high, divideby=divideby[1], cutoff=cutoff[1])
    div1_low_div2_high, div1_low_div2_low, _ = divide_samples(div1_low, divideby=divideby[1], cutoff=cutoff[1])
    
    return div1_high_div2_high, div1_high_div2_low, div1_low_div2_high, div1_low_div2_low

In [ ]:
from lifelines import KaplanMeierFitter, CoxPHFitter
from lifelines.statistics import logrank_test

def logrank_p_value(data1, data2, event="OS", time="OS.time"):
    """
    calculate log rank p values
    """
    result = logrank_test(data1[time], data2[time],
                          data1[event], data2[event])
    return result.p_value

def cox_regression_p_value(survival_data, formula='age + gender + stage + race + IGSF8', event="OS", time="OS.time"): #   + cbioportal.subtype
    """
    Fit a cox regression model. Age, gender and stage are considered as covariates.
    """
    cph = CoxPHFitter(penalizer=0.5)#
    cols = formula.split(" + ") + [time, event]
    tmp = survival_data[cols].dropna()
    cph.fit(tmp, formula=formula, duration_col=time, event_col=event, robust=True)
    
    return cph.summary.loc["IGSF8", "p"], cph

In [ ]:
# Calculate HR for all cohorts. Similar to the previous codes, but plotting is excluded.

coxs = []

for pid in [
    'KIRP', 'PAAD', 'BRCA', 'READ', 'UCEC',  
    'TGCT', 'THCA',  'KIRC', 'COAD',
    'STAD', 'SKCM', 'LUSC', 'HNSC', 'CESC', 'LIHC', 'LUAD', 'BLCA',
           ]:
    
    survival_subdata = expr_data.query('project_id==@pid')
    survival_subdata = survival_subdata[survival_subdata["OS.time"].notna()]
    div1_high, div1_low, _ = divide_samples(survival_subdata, divideby="IGSF8", cutoff=".75")
    
    # hypothesis testing for IGSF8-high and IGSF8-low groups within the MHCI-high population
    p_logrank = logrank_p_value(div1_high, div1_low, event="OS", time="OS.time")
    p_cox1, cox_res1 = cox_regression_p_value(pd.concat([div1_high, div1_low]), 
                                      formula='age + gender + stage + race + IGSF8', #
                                      event="OS", time="OS.time")
    p_cox2, cox_res2 = cox_regression_p_value(pd.concat([div1_high, div1_low]), 
                                      formula='age + gender + stage + race + A03 + A11 + IGSF8', #
                                      event="OS", time="OS.time")
    
    cox_res1 = cox_res1.summary.loc['IGSF8', ['exp(coef)', 'exp(coef) lower 95%', 'exp(coef) upper 95%', 'p']].to_frame("A*03/A*11 uncorrected").T
    cox_res1['sample_size'] = survival_subdata.shape[0]

    cox_res2 = cox_res2.summary.loc['IGSF8', ['exp(coef)', 'exp(coef) lower 95%', 'exp(coef) upper 95%', 'p']].to_frame("A*03/A*11 corrected").T
    cox_res2['sample_size'] = survival_subdata.shape[0]
    
    cox = pd.concat([cox_res1, cox_res2], axis=0)
    cox['project'] = pid
        
    coxs.append(cox)
    
coxs = pd.concat(coxs)

In [ ]:
coxs

In [ ]:
# target the cohorts of significance
coxs = coxs.query('sample_size>20')
coxs = coxs.rename_axis("HLA_adjusted").reset_index()
coxs['ci'] = (coxs['exp(coef) upper 95%'] - coxs['exp(coef) lower 95%']) / 2

# manipulate the dataframe
coxs1 = coxs.set_index(["project", "HLA_adjusted"]).unstack()
coxs1.columns = coxs1.columns.swaplevel()
coxs1 = coxs1.sort_index(axis=1)

MHCI_high_HR = coxs1[['A*03/A*11 uncorrected']].sort_values(("A*03/A*11 uncorrected", "exp(coef)")).reset_index()
MHCI_low_HR = coxs1[['A*03/A*11 corrected']].reindex(MHCI_high_HR['project']).reset_index()

In [ ]:
# set markers for different groups, MHCI-high and MHCI-low

errorbar_kwargs_low = {"c": cmap.npg_palette(3),
                       "fmt": "s",
                       "markersize": 3,
                       "markerfacecolor": cmap.npg_palette(3),
                       "markeredgewidth": .5,
                       "capsize": 1.,
                       "elinewidth": .5
                      }

errorbar_kwargs_high = {"c": cmap.npg_palette(0),
                        "fmt": "s",
                        "markersize": 3,
                        "markerfacecolor": cmap.npg_palette(0),
                        "markeredgewidth": .5,
                        "capsize": 1.,
                        "elinewidth": .5
                       }

In [ ]:
# plotting

from matplotlib.patches import Patch

plt.figure(figsize = (2.5, 3.2));
plt.errorbar(MHCI_low_HR['A*03/A*11 corrected']["exp(coef)"], MHCI_low_HR.index + .17, xerr=MHCI_low_HR['A*03/A*11 corrected']["ci"], **errorbar_kwargs_low);
plt.errorbar(MHCI_high_HR['A*03/A*11 uncorrected']["exp(coef)"], MHCI_high_HR.index - .17, xerr=MHCI_high_HR['A*03/A*11 uncorrected']["ci"], **errorbar_kwargs_high);

[plt.text(row["exp(coef) upper 95%"] + .05, idx + .1, f"p={row['p']:.3f}", fontsize=4, c=cmap.npg_palette(3)) for idx, row in MHCI_low_HR['A*03/A*11 corrected'].iterrows() if row['p']<.05]
[plt.text(row["exp(coef) upper 95%"] + .05, idx - .2, f"p={row['p']:.3f}", fontsize=4, c=cmap.npg_palette(0)) for idx, row in MHCI_high_HR['A*03/A*11 uncorrected'].iterrows() 
 if row['p']<.05 and row["exp(coef) upper 95%"]<5]

[plt.text(row["exp(coef) upper 95%"] - 2, idx + .05, f"p={row['p']:.3f}", fontsize=4, c=cmap.npg_palette(0)) for idx, row in MHCI_high_HR['A*03/A*11 uncorrected'].iterrows() 
 if row['p']<.05 and row["exp(coef) upper 95%"]>5]

plt.vlines(1, -1., 16.5, ls="--", lw=.5, color='gray')
plt.yticks(MHCI_high_HR.index, MHCI_high_HR['project'].values, fontsize=6)
plt.ylim(-.5, 16.5)
# plt.xlim(-3.5, 20.)
plt.xlabel("Hazard Ratio for IGSF8 (95% CI)", fontsize=6)

handles = [
    Patch(edgecolor=color, facecolor=color, label=label) 
    for label, color in zip(['A*03/A*11 corrected', 'A*03/A*11 uncorrected'], [cmap.npg_palette(3), cmap.npg_palette(0)])
]
plt.legend(handles=handles, loc=(-0.01, 1.02), ncols=2, fontsize=5, frameon=False);

plt.savefig("../figures/FigS3X.pdf", bbox_inches = "tight")